In [14]:
import pandas as pd

#Ładowanie danych
df = pd.read_excel('/content/RYNEK_NIER.xlsx', header=25)


df['cena_numeric'] = pd.to_numeric(df[6212], errors='coerce')

# Group by 'rynek pierwotny' (transakcje) and count zeros in 'cena_numeric'
zeros_by_transakcja = df.groupby('rynek pierwotny')['cena_numeric'].apply(lambda x: (x == 0).sum())

# Display the results
print("Number of zeros in 'cena' (column 6212) for each 'transakcje' (column rynek pierwotny) option:")
display(zeros_by_transakcja)

# Identify the option with the most zeros
option_with_most_zeros = zeros_by_transakcja.idxmax()
max_zeros_count = zeros_by_transakcja.max()

print(f"\nOption in 'rynek pierwotny' with the most zeros in 'cena' (column 6212): {option_with_most_zeros} (count: {max_zeros_count})")

DataFrame columns:
Index(['POLSKA', 'rynek pierwotny', '2020', 6212], dtype='object')
Number of zeros in 'cena' (column 6212) for each 'transakcje' (column rynek pierwotny) option:


,cena_numeric
rynek pierwotny,
ogółem,106
rynek pierwotny,1454
rynek wtórny,109



Option in 'rynek pierwotny' with the most zeros in 'cena' (column 6212): rynek pierwotny (count: 1454)


In [15]:
print (df)

                      POLSKA  rynek pierwotny  2020   6212  cena_numeric
0                     POLSKA  rynek pierwotny  2021   6899        6899.0
1                     POLSKA  rynek pierwotny  2022   7466        7466.0
2                     POLSKA  rynek pierwotny  2023   8155        8155.0
3                     POLSKA     rynek wtórny  2010   4069        4069.0
4                     POLSKA     rynek wtórny  2011   4039        4039.0
...                      ...              ...   ...    ...           ...
16644  Powiat m. Świnoujście     rynek wtórny  2019   7345        7345.0
16645  Powiat m. Świnoujście     rynek wtórny  2020   8200        8200.0
16646  Powiat m. Świnoujście     rynek wtórny  2021   8751        8751.0
16647  Powiat m. Świnoujście     rynek wtórny  2022  11131       11131.0
16648  Powiat m. Świnoujście     rynek wtórny  2023  11196       11196.0

[16649 rows x 5 columns]


In [16]:
# Aby wyświetlić nazwy wszystkich kolumn w DataFrame 'df':
print(df.columns)

Index(['POLSKA', 'rynek pierwotny', '2020', 6212, 'cena_numeric'], dtype='object')


In [ ]:
import pandas as pd

# Load the Excel file into a pandas DataFrame, treating the very first row (index 0) as the header
df = pd.read_excel('/content/RYNEK_NIER.xlsx', header=0)

# Display the first 5 rows of the DataFrame to see the effect
display(df.head())